about 1-2h

In [1]:
#!/usr/bin/env python
from scipy.signal import hilbert
from scipy.signal import welch
from scipy import signal
import pandas as pd
from mne.time_frequency import psd_array_multitaper, psd_array_welch
from fooof import FOOOF
import numpy as np
import argparse
import mne.io
import mne
import os
import neurokit2 as nk
import multiprocessing as mp
from scipy.io import loadmat
from scipy.io import savemat
import glob
from joblib import Parallel, delayed
from tqdm import tqdm

mne.set_log_level("ERROR")



# call:  python features_DFA.py -data_dir EPOCHS -output_dir RESULTS -part_info EPOCHS/participants.txt -lfrequ 8 -hfrequ 14

def get_channel_hurst(ch_data,sfreq):

    scale = nk.expspace(1*sfreq, 20*sfreq, 40, base=2).astype(np.int64)

    analytic_signal = hilbert(ch_data)
    amplitude_envelope = np.abs(analytic_signal)

    try:
        hurst_fh, _ = nk.fractal_hurst(amplitude_envelope, scale=scale, show=False)
    except:
        hurst_fh = float('nan')

    try:
        hurst_dfa, _ = nk.fractal_dfa(amplitude_envelope, scale=scale, show=False)
    except:
        hurst_dfa = float('nan')

    return  hurst_fh, hurst_dfa



In [2]:
# Define input and output directories
out_dir = 'data/output/DFA/'
in_dir = 'data/input/continuous/'

fbands = [[1,45], [1, 4], [4, 8], [8, 13], [13, 30], [30, 45]]
# fbands = [[1,45]]

for fband in fbands:
    lfreq = fband[0]
    hfreq = fband[1]

    # Ensure the output directory exists
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    # Find all .fif files
    paths = glob.glob(in_dir + '*.fif')
    paths.sort()
    
    # Lists to store results
    HURST_FH = []
    HURST_DFA = []
    sub_ids = []
    days = []
    conditions = []

    # Loop over each file

    for path_i, path in enumerate(paths):
        basename = os.path.basename(path)
        sub = basename[:4]
        day = basename[5:9]
        condition = basename[10:-8]
        if len(basename)==36:
            condition = basename[10:15]+basename[-5:-4]

        sub_ids.append(sub)
        days.append(day)
        conditions.append(condition)

        # Load data
        raw = mne.io.read_raw_fif(path, preload=True)
        raw.interpolate_bads(reset_bads=True)
        data = raw.get_data()[0:32, :]  # Use only first 32 channels
        fs = 256  # Sampling frequency
        sig_length = min(data.shape[1] / fs, 900)  # Ensure max 200s of data
        # new: 20 min == 1200s
        nr_channels = data.shape[0]  # Number of channels

        # Cut data
        cut = int(sig_length * fs)
        data = data[:, :cut]

        # Apply filtering
        data_filt = mne.filter.filter_data(data, sfreq=fs, l_freq=lfreq, h_freq=hfreq, verbose=False)

        # Run multiprocessing using joblib
        results = Parallel(n_jobs=-1)(delayed(get_channel_hurst)(data_filt[ch, :], fs) for ch in range(nr_channels))

        # Store results
        HURST_FH.append(np.mean(pd.DataFrame(results)[0]))
        HURST_DFA.append(np.mean(pd.DataFrame(results)[1]))

    # Save results with metadata
    output_df = pd.DataFrame({
        'sub': sub_ids,
        'day': days,
        'condition': conditions,
        'HURST_FH': HURST_FH,
        'HURST_DFA': HURST_DFA
    })

    # Save to CSV
    output_df.to_csv(f'{out_dir}/DFA_{lfreq}_{hfreq}.csv', index=False, sep=',')

    print("Processing complete. Results saved.")


/Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/venv/lib/python3.8/site-packages/neurokit2/complexity/fractal_dfa.py:415: UserWarning: All detrended segments have no variance. Retuning NaN.
  warn("All detrended segments have no variance. Retuning NaN.")
/Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/venv/lib/python3.8/site-packages/neurokit2/complexity/fractal_dfa.py:415: UserWarning: All detrended segments have no variance. Retuning NaN.
  warn("All detrended segments have no variance. Retuning NaN.")
/Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/venv/lib/python3.8/site-packages/neurokit2/complexity/fractal_dfa.py:415: UserWarning: All detrended segments have no variance. Retuning NaN.
  warn("All detrended segments have no variance. Retuning NaN.")
/Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/venv/lib/python3.8/site-packages/neurokit2/complexity/fractal_dfa.py:415: UserWarning: All detrended segments have no v

Processing complete. Results saved.
Processing complete. Results saved.
Processing complete. Results saved.
Processing complete. Results saved.
Processing complete. Results saved.
Processing complete. Results saved.
